In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

pd.options.display.max_columns = 1000
plt.style.use('default')

### Read 2018 demographic data

In [2]:
dfc = pd.read_csv('../files/census/cc-est2018-alldata.csv', encoding='ISO-8859-1')

In [3]:
dfc.YEAR.unique()[-1]

11

In [4]:
# Get 2010 data
dfc = dfc.loc[dfc.YEAR == dfc.YEAR.unique()[-1]]

In [5]:
start_race_idx = dfc.columns.tolist().index('WA_MALE')

In [6]:
race_cols = dfc.columns[start_race_idx:]

In [7]:
# COMBINE MALE AND FEMALE RACE COLUMNS
for col in race_cols:
    if col[-6:] == 'FEMALE':
        identifier = col[:-7]
        col_male = identifier + '_MALE'
        dfc[identifier] = dfc[identifier + '_FEMALE'] + dfc[identifier + '_MALE']


In [8]:
dfc.drop(columns=race_cols, inplace=True)

In [9]:
# GET AGE BRACKETS FOR TOTAL POPULATION, USE TOTAL FOR ALL OTHER DEMOGRAPHICS
dfage = pd.DataFrame(dfc.groupby(['STATE', 'COUNTY', 'AGEGRP']).TOT_POP.sum()).reset_index()

dfage = pd.pivot_table(dfage, index=['STATE', 'COUNTY'],
               values='TOT_POP', columns='AGEGRP').reset_index(drop=True)

dfc = dfc.loc[dfc.AGEGRP == 0]

dfc = dfc.reset_index(drop=True)

dfage.rename(columns={0:'total'}, inplace=True)

(dfage.total == dfc.TOT_POP).unique()

array([ True])

In [10]:
dfc = pd.concat([dfc, dfage], axis=1)

In [11]:
dfc = dfc.drop(columns=['SUMLEV'])

In [12]:
dfc = dfc.rename(columns={1:'zero_four', 2:'five_nine', 3:'ten_fourteen', 4:'fifteen_nineteen',
                   5:'twenty_twentyfour', 6:'twentyfive_twentynine', 7:'thirty_thirtyfour',
                   8:'thirtyfive_thirtynine', 9:'forty_fortyfour', 10:'fortyfive_fortynine',
                   11:'fifty_fiftyfour', 12:'fiftyfive_fiftynine', 13:'sixty_sixtyfour',
                   14:'sixtyfive_sixtynine', 15:'seventy_seventyfour', 
                    16:'seventyfive_seventynine', 17:'eighty_eightyfour', 18:'eightyfive_older'})

In [13]:
dfc = dfc.drop(columns=['total', 'AGEGRP', 'YEAR'])

In [14]:
# NORMALIZE FEATURES TO POPULATION SIZE
dont_norm = []
for column in dfc.columns[5:]:
    if column not in dont_norm:
        dfc[column] /= dfc.TOT_POP

In [15]:
dfc['FIPS'] = dfc.STATE.map(lambda x: '{:02d}'.format(x)) + dfc.COUNTY.map(lambda x: 
                                                                        '{:03d}'.format(x))

### Read 2018 population data

In [16]:
dfp = pd.read_csv('../files/census/co-est2018-alldata.csv', encoding='ISO-8859-1')

In [17]:
dfp['FIPS'] = dfp.STATE.map(lambda x: '{:02d}'.format(x)) + dfp.COUNTY.map(lambda x: 
                                                                        '{:03d}'.format(x))

In [18]:
keep_columns = ['RNETMIG2018', 'RDOMESTICMIG2018', 'RINTERNATIONALMIG2018',
               'RNATURALINC2018', 'RDEATH2018', 'RBIRTH2018',
               'REGION', 'DIVISION', 'FIPS']

In [19]:
divide_columns = ['GQESTIMATES2018', 'NPOPCHG_2018']

In [20]:
for col in divide_columns:
    dfp[col] = dfp[col] / dfp.POPESTIMATE2018

In [21]:
dfp = dfp[keep_columns + divide_columns]

In [22]:
df = dfc.set_index('FIPS').join(dfp.set_index('FIPS'))

### Read 2018 employment and income data

In [23]:
dfe = pd.read_csv('../files/census/unemployment.csv', encoding='ISO-8859-1', header=7)

In [24]:
dfe_keep = ['FIPStxt', 'Civilian_labor_force_2018', 'Employed_2018', 'Unemployed_2018', 
            'Median_Household_Income_2018', 'Rural_urban_continuum_code_2013',
           'Urban_influence_code_2013', 'Metro_2013']

In [25]:
dfe = dfe[dfe_keep]

In [26]:
with_commas = ['Civilian_labor_force_2018', 'Employed_2018', 'Unemployed_2018', 
            'Median_Household_Income_2018']

In [27]:
for col in with_commas:
    dfe[col] = dfe[col].str.replace(',', '').fillna(0).astype(int).replace(0, np.nan)

#### Some counties in Alaska have missing values

In [28]:
dfe['Employed_2018'].sort_values()[-5:]

194     18460725.0
0      155102319.0
92             NaN
95             NaN
99             NaN
Name: Employed_2018, dtype: float64

In [29]:
dfe.loc[dfe['Employed_2018'].isnull()]

,FIPStxt,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Median_Household_Income_2018,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013
92,2201,NaN,NaN,NaN,NaN,NaN,NaN,0.0
95,2232,NaN,NaN,NaN,NaN,NaN,NaN,0.0
99,2280,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [30]:
dfe['FIPStxt'] = dfe.FIPStxt.map(lambda x: '{:05d}'.format(x))

In [31]:
df_temp = df.join(dfe.set_index('FIPStxt'))

In [32]:
new_divide = ['Civilian_labor_force_2018', 'Employed_2018', 'Unemployed_2018']

In [33]:
for col in new_divide:
    df_temp[col] = df_temp[col] / df_temp.TOT_POP

In [34]:
df = df_temp

In [35]:
# SET EMPLOYMENT AND UNEMPLOYMENT RATES TO BE PERCENT OF LABOR FORCE
df['Employed_2018'] = df['Employed_2018'] / df['Civilian_labor_force_2018']
df['Unemployed_2018'] = df['Unemployed_2018'] / df['Civilian_labor_force_2018']

### Read 2018 education data

In [36]:
dfd = pd.read_csv('../files/census/education.csv', encoding='ISO-8859-1', header=4)

In [37]:
dfd_keep = ['FIPS Code', 'Percent of adults with less than a high school diploma, 2014-18',
           'Percent of adults with a high school diploma only, 2014-18',
           'Percent of adults completing some college or associate\'s degree, 2014-18',
           'Percent of adults with a bachelor\'s degree or higher, 2014-18']

In [38]:
dfd = dfd[dfd_keep]

In [39]:
dfd['FIPS Code'] = dfd['FIPS Code'].map(lambda x: '{:05d}'.format(x))

In [40]:
df = df.join(dfd.set_index('FIPS Code'))

### Read land area data

In [41]:
dfl = pd.read_csv('../files/land.csv')

In [42]:
dfl.fips = dfl.fips.map(lambda x: '{:05d}'.format(x))

In [43]:
dfl = dfl.drop('POP060210', axis=1)

In [44]:
dfl.rename(columns={'LND110210':'LAND_AREA'}, inplace=True)

In [45]:
df = df.join(dfl.set_index('fips'))

In [46]:
df['DENSITY'] = df.TOT_POP / df.LAND_AREA

In [47]:
df

,STATE,COUNTY,STNAME,CTYNAME,TOT_POP,TOT_MALE,TOT_FEMALE,WA,BA,IA,AA,NA,TOM,WAC,BAC,IAC,AAC,NAC,NH,NHWA,NHBA,NHIA,NHAA,NHNA,NHTOM,NHWAC,NHBAC,NHIAC,NHAAC,NHNAC,H,HWA,HBA,HIA,HAA,HNA,HTOM,HWAC,HBAC,HIAC,HAAC,HNAC,zero_four,five_nine,ten_fourteen,fifteen_nineteen,twenty_twentyfour,twentyfive_twentynine,thirty_thirtyfour,thirtyfive_thirtynine,forty_fortyfour,fortyfive_fortynine,fifty_fiftyfour,fiftyfive_fiftynine,sixty_sixtyfour,sixtyfive_sixtynine,seventy_seventyfour,seventyfive_seventynine,eighty_eightyfour,eightyfive_older,RNETMIG2018,RDOMESTICMIG2018,RINTERNATIONALMIG2018,RNATURALINC2018,RDEATH2018,RBIRTH2018,REGION,DIVISION,GQESTIMATES2018,NPOPCHG_2018,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Median_Household_Income_2018,Rural_urban_continuum_code_2013,Urban_influence_code_2013,Metro_2013,"Percent of adults with less than a high school diploma, 2014-18","Percent of adults with a high school diploma only, 2014-18","Percent of adults completing some college or associate's degree, 2014-18","Percent of adults with a bachelor's degree or higher, 2014-18",LAND_AREA,DENSITY
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01001,1,1,Alabama,Autauga County,55601,0.485513,0.514487,0.767252,0.196309,0.004802,0.012248,0.001115,0.018273,0.783799,0.206165,0.010270,0.017302,0.002104,0.970342,0.743080,0.193432,0.004370,0.012050,0.000773,0.016636,0.758296,0.202388,0.009029,0.016654,0.001493,0.029658,0.024172,0.002878,0.000432,0.000198,0.000342,0.001637,0.025503,0.003777,0.001241,0.000647,0.000611,0.060503,0.061564,0.069819,0.067535,0.057067,0.066635,0.062247,0.065251,0.065035,0.071330,0.068542,0.070628,0.058218,0.046564,0.041564,0.031276,0.020557,0.015665,0.630381,0.666403,-0.036022,2.215338,9.581787,11.797125,3,6,0.008183,0.002842,0.471143,0.964308,0.035692,59338.0,2.0,2.0,1.0,11.3,32.6,28.4,27.7,594.44,93.535092
01003,1,3,Alabama,Baldwin County,218022,0.484616,0.515384,0.872852,0.089404,0.007724,0.011503,0.000670,0.017847,0.889598,0.096885,0.015939,0.015613,0.001624,0.953532,0.831113,0.087840,0.006701,0.011343,0.000509,0.016026,0.846300,0.094536,0.013870,0.015035,0.001211,0.046468,0.041739,0.001564,0.001023,0.000161,0.000161,0.001821,0.043298,0.002348,0.002069,0.000578,0.000413,0.054834,0.058719,0.064127,0.059315,0.050949,0.057201,0.055302,0.060343,0.059063,0.066328,0.065324,0.072213,0.071851,0.065438,0.057201,0.038826,0.023227,0.019741,24.846682,24.298662,0.548020,0.120750,10.347366,10.468116,3,6,0.010045,0.024782,0.436805,0.964046,0.035954,57588.0,3.0,2.0,1.0,9.7,27.6,31.3,31.3,1589.78,137.139730
01005,1,5,Alabama,Barbour County,24881,0.527832,0.472168,0.490696,0.483984,0.006591,0.004542,0.001849,0.012339,0.500342,0.492906,0.011575,0.006069,0.002130,0.957236,0.456413,0.480326,0.003416,0.004542,0.001085,0.011455,0.465456,0.488606,0.007837,0.006029,0.001367,0.042764,0.034283,0.003657,0.003175,0.000000,0.000764,0.000884,0.034886,0.004300,0.003738,0.000040,0.000764,0.051606,0.058197,0.060649,0.058318,0.059162,0.072184,0.067119,0.061533,0.058277,0.065271,0.064427,0.067320,0.061734,0.061895,0.053736,0.036454,0.023150,0.018970,-8.593297,-9.072923,0.479626,-2.518036,12.949899,10.431863,3,6,0.113339,-0.011133,0.338170,0.949251,0.050749,34382.0,6.0,6.0,0.0,27.0,35.7,25.1,12.2,884.88,28.117937
01007,1,7,Alabama,Bibb County,22400,0.532188,0.467813,0.768348,0.212946,0.004375,0.002366,0.001161,0.010804,0.778125,0.219018,0.008973,0.003795,0.001741,0.973750,0.745893,0.211205,0.003839,0.002321,0.000268,0.010223,0.755179,0.216920,0.008036,0.003750,0.000804,0.026250,0.022455,0.001741,0.000536,0.000045,0.000893,0.000580,0.022946,0.002098,0.000937,0.000045,0.000937,0.057366,0.052277,0.059196,0.056696,0.058170,0.077054,0.071473,0.067500,0.064286,0.071339,0.070580,0.068482,0.060848,0.052768,0.044286,0.032634,0.018839,0.016205,-6.584362,-7.029251,0.444889,-0.266934,11.389167,11.122233,3,6,0.096027,-0.006920,0.384152,0.960837,0.039163,46064.0,1.0,1.0,1.0,16.8,47.3,24.4,11.5,622.58,35.979312
01009,1,9,Alabama,

In [48]:
dfdc = pd.read_csv('../files/census/dc.csv')

In [49]:
dfdc.county = dfdc.county.map(lambda x: x[6:])

In [50]:
df = df.join(dfdc[['county', 'med_age']].set_index('county'))

In [51]:
df = df.loc[~(df.CTYNAME == 'Kalawao County')]

### Get 2016 election results

In [52]:
# dfel = pd.read_csv('../files/election/2016_US_County_Level_Presidential_Results.csv', index_col=0)

# dfel.combined_fips = dfel.combined_fips.map(lambda x: '{:05d}'.format(x))

# dfel = dfel[['combined_fips', 'per_dem', 'per_gop']]

# df = df.join(dfel.set_index('combined_fips'))

### Save census data to disk

In [53]:
df.to_csv('../files/modeling_data/census_data.csv')